Subset.py

In [26]:
import pandas as pd

def subset(file_paths=[]):

    dataframes = [pd.read_csv(file) for file in file_paths]
    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dataset.py

In [27]:
from sklearn.model_selection import train_test_split
def dataset():
    file_paths = [
        '/content/drive/MyDrive/combined_features_with_date1.csv',
        # '/content/drive/MyDrive/combined_features_with_date2.csv',
        # '/content/drive/MyDrive/combined_features_with_date3.csv'
    ]

    Dataset = subset(file_paths=file_paths)

    TrainingData, TestData = train_test_split(Dataset, test_size=0.2, random_state=42)

    TrainingData.to_csv("TrainingData.csv", index=False)
    TestData.to_csv("TestData.csv", index=False)

    with open("features_info.txt", "w") as f:
        f.write("Features used:\n")
        for feature in Dataset.columns:
            f.write(f"{feature}\n")

    print("Datasets have been successfully split and saved as 'TrainingData.csv' and 'TestData.csv'.")
    print("Features info saved as 'features_info.txt'.")

    return TrainingData, TestData


neuralnet.py

In [29]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os

def build_model(input_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_size,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def prepare_dataset(data, batch_size=32, shuffle=True):
    # Separate features and target
    X = data.drop(columns=["streamflow"]).values
    y = data["streamflow"].values

    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Create tf.data.Dataset from the arrays
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # Shuffle and batch the dataset
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X))
    dataset = dataset.batch(batch_size)

    return dataset, scaler

def train_nn(TrainingData, epochs=10, batch_size=32, checkpoint_dir="/content/checkpoints"):
    # Ensure checkpoint directory exists
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Update checkpoint path to end in `.weights.h5`
    checkpoint_path = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}.weights.h5")
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        save_best_only=True, # Save only the best model
        monitor='val_loss',   # Monitor validation loss
        mode='min',           # Save model with minimum validation loss
        verbose=1
    )

    # Remove unnecessary columns and NaNs
    if 'date' in TrainingData.columns:
        TrainingData = TrainingData.drop(columns=['date'])
    TrainingData = TrainingData.dropna()

    # Prepare the dataset
    train_dataset, scaler = prepare_dataset(TrainingData, batch_size=batch_size, shuffle=True)

    # Build the model
    input_size = TrainingData.drop(columns=["streamflow"]).shape[1]
    model = build_model(input_size)

    # Train the model with checkpointing
    model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=train_dataset.take(20),  # Small validation split
        callbacks=[checkpoint_callback],
        verbose=1
    )

    print("Model training complete.")
    return model, scaler

if __name__ == "__main__":
    # Load training data
    # train_data, validation_data, test_data = dataset()
    train_data = pd.read_csv('/content/TrainingData.csv').sample(10000, random_state=1)
    NNModel, scaler = train_nn(train_data)


Epoch 1/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 7.0806 - mae: 1.6215
Epoch 1: val_loss improved from inf to 8.09063, saving model to /content/checkpoints/model_epoch_01.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 7.9026 - mae: 1.6776 - val_loss: 8.0906 - val_mae: 1.6326
Epoch 2/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 5.8985 - mae: 1.5911
Epoch 2: val_loss improved from 8.09063 to 7.47118, saving model to /content/checkpoints/model_epoch_02.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 7.1574 - mae: 1.6028 - val_loss: 7.4712 - val_mae: 1.5684
Epoch 3/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 1.6809 - mae: 0.9391
Epoch 3: val_loss improved from 7.47118 to 6.94973, saving model to /content/checkpoints/model_epoch_03.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 5.4538 - mae: 1.3486 - val_loss: 6.9497 - val_mae: 1.5183
Epoch 4/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.3910 - mae: 1.4315
Epoch 4: val_loss improved from 6.949